In [22]:
import os
import glob
import numpy as np
import faiss
import torch
import pandas as pd
from torch import Tensor
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from langchain.text_splitter import RecursiveCharacterTextSplitter

FOLDER_PATH = "../md_benchmark/benchmark/Руководства к РФ ПО"
BENCHMARCK_PATH = "../benchmark/benchmark.csv"
RETRIEVER = "intfloat/multilingual-e5-large"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# !ls {FOLDER_PATH}

In [ ]:
def read_md_files(folder_path):
    files = glob.glob(os.path.join(folder_path, "*.md"))
    documents = []
    for file_path in files:
        with open(file_path, "r", encoding="utf-8") as f:
            doc_name = os.path.splitext(os.path.basename(file_path))[0]
            documents.append((doc_name, f.read()))
    return documents

In [4]:
documents = read_md_files(FOLDER_PATH)
len(documents)

2

In [5]:
def split_documents(documents):
    tokenizer = AutoTokenizer.from_pretrained(RETRIEVER)
    
    def token_counter(text):
        return len(tokenizer.encode(text, add_special_tokens=False))
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=50,
        length_function=token_counter,
        separators=["\n\n", "\n", "."]
    )
    
    chunks = []
    for doc_name, content in documents:
        for chunk in text_splitter.split_text(content):
            chunks.append((doc_name, chunk))
    return chunks

In [6]:
chunks = split_documents(documents)
print(f"Создано {len(chunks)} чанков")

Token indices sequence length is longer than the specified maximum sequence length for this model (1207 > 512). Running this sequence through the model will result in indexing errors


Создано 715 чанков


In [7]:
print(chunks[123])

('1c', 'Раскрыть группу в списке можно одним из следующих способов:\n\n- \uf0b7 дважды  щелкнуть  мышью  на  строке  списка  с  наименованием нужной группы;\n- \uf0b7 нажать левую кнопку мыши  на символе [+] в строке с наименованием нужной группы;\n- \uf0b7 установить курсор на строке с наименованием нужной группы и нажать клавиши Ctrl + Стрелка вниз ;\n- \uf0b7 установить курсор на строку с наименованием группы и нажать кнопку + на цифровой клавиатуре;\n- \uf0b7 установить курсор на строку с наименованием нужной группы и выбрать пункт Все действия - Перейти на уровень ниже .\n\nДля возврата на предыдущий уровень можно нажать клавиши Ctrl + Стрелка вверх ,  находясь в любой строке группы, или выбрать пункт Все действия - Перейти на уровень выше .\n\nРежим «Дерево» .  Если  в  качестве  режима  просмотра  выбран  режим Дерево , то элементы списка отображаются в виде дерева.\n\n<!-- image -->')


In [8]:
print(chunks[124])

('1c', 'Режим «Дерево» .  Если  в  качестве  режима  просмотра  выбран  режим Дерево , то элементы списка отображаются в виде дерева.\n\n<!-- image -->\n\nДля  удобства работы  дерево имеет возможность  раскрываться  и закрываться. Знак + (плюс) в узле ветви указывает, что ветвь можно раскрыть.  При  нажатии  левой  кнопки  мыши  на  этом  знаке  ветвь откроет для просмотра следующий уровень, а знак + (плюс) изменится на -(минус). Свернуть ветвь дерева можно нажатием мыши на знаке -(минус).\n\nЕсли дерево раскрыто, то для перехода к нужной группе используются клавиши Стрелка Вверх и Стрелка Вниз .\n\nДля одновременного раскрытия и перехода на группу нижнего уровня следует  использовать  сочетание  клавиш Ctrl + Стрелка  Вниз. Для перехода к родительской группе Ctrl + Стрелка Вверх .\n\nЧтобы свернуть узел дерева и все подчиненные, используются клавиши Shift + Alt + Num-. Чтобы свернуть все узлы дерева, используются  клавиши Ctrl + Shift + Alt + Num-. Чтобы  развернуть узел дерева и все

In [ ]:
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

def create_embeddings(chunks, batch_size=8, is_queries=False):
    tokenizer = AutoTokenizer.from_pretrained(RETRIEVER)
    model = AutoModel.from_pretrained(RETRIEVER)
    model.eval()

    model.to(DEVICE)

    embeddings = []

    for i in tqdm(range(0, len(chunks), batch_size)):
        batch = chunks[i:i+batch_size]

        if is_queries:
            prefixed_batch = [f"{doc_name}: {text}" for doc_name, text in batch]
        else:
            prefixed_batch = batch

        inputs = tokenizer(
            prefixed_batch,
            padding=True,
            truncation=True,
            return_tensors="pt",
            max_length=512
        )

        inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)

        batch_embeddings = average_pool(outputs.last_hidden_state, inputs['attention_mask'])
        
        batch_embeddings = torch.nn.functional.normalize(batch_embeddings, p=2, dim=1)
        embeddings.append(batch_embeddings.cpu().numpy())
    
    return np.concatenate(embeddings, axis=0)

In [12]:
embeddings = create_embeddings(chunks, batch_size=32)


100%|██████████| 23/23 [00:11<00:00,  2.07it/s]


In [13]:
embeddings.shape

(715, 1024)

In [16]:
def create_faiss_index(embeddings):
    folder_name = os.path.basename(FOLDER_PATH.rstrip("/\\"))
    index_path = f"faiss_index_{folder_name}.index"

    dimension = embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)
    index.add(embeddings.astype(np.float32))
    faiss.write_index(index, index_path)
    return index

In [18]:
index = create_faiss_index(embeddings)

In [20]:
torch.cuda.empty_cache()

In [33]:
def read_benchmark(set):
    df = pd.read_csv(BENCHMARCK_PATH)
    df = df[df["Сет документов"] == set].reset_index(drop=True)
    return df

In [34]:
df = read_benchmark(os.path.basename(FOLDER_PATH.rstrip("/\\")))

In [35]:
df

,Домен документов,Сет документов,Название документа,Отрывок из документа,Тип вопроса,Вопрос,Ответ
0,Техническая документация,Руководства к РФ ПО,1c.pdf,Функционирование системы «1С:Предприятие» дели...,Simple,Как делится функционирование системы «1С:Предп...,Функционирование системы «1С:Предприятие» дели...
1,Техническая документация,Руководства к РФ ПО,1c.pdf,Функционирование системы «1С:Предприятие» дели...,With errors,"Как фукционерует системма «1С:Преприятие», на ...",Функционирование системы «1С:Предприятие» дели...
2,Техническая документация,Руководства к РФ ПО,1c.pdf,Функционирование системы «1С:Предприятие» дели...,Trash,"Привет! Мне надо понять, как именно работает э...",Функционирование системы «1С:Предприятие» дели...
3,Техническая документация,Руководства к РФ ПО,1c.pdf,Функционирование системы «1С:Предприятие» дели...,Reformulation,Какие ключевые стадии определяют процесс работ...,Функционирование системы «1С:Предприятие» дели...
4,Техническая документация,Руководства к РФ ПО,1c.pdf,Функционирование системы «1С:Предприятие» дели...,Incorrect by design,В «1С:Предприятие» пользователь сначала работа...,"Нет, наоборот. Функционирование системы «1С:Пр..."
...,...,...,...,...,...,...,...
72,Техническая документация,Руководства к РФ ПО,1c.pdf,Если один и тот же объект пытаются отредактиро...,Logical thinking,"Можно ли провести документ, если он заблокиров...",Нет. Если документ заблокирован другим пользов...
73,Техническая документация,Руководства к РФ ПО,1c.pdf,Установленные значения настроек сохраняются ме...,Logical thinking,"Как сохранить настройки отчета, чтобы использо...",Настройки отчета сохраняются через меню Все де...
74,Техническая документация,Руководства к РФ ПО,mysql.pdf,"При первичном запуске, необходимо ввести парол...",Logical thinking,Как изменить пароль главного администратора си...,Пароль главного администратора изменяется чере...
75,Техническая документация,Руководства к РФ ПО,mysql.pdf,Единовременное назначение пользователю только ...,Logical thinking,Можно ли назначить один прибор нескольким поль...,Нет. Программа позволяет назначить только один...


In [ ]:
def search(queries, k = 5):
    query_embeddings = create_embeddings(queries, is_queries=True)
    scores, indices = index.search(query_embeddings.astype(np.float32), k)
    
    results = []
    for query, query_scores, query_indices in zip(queries, scores, indices):
        result = {
            "query": query,
            "results": [
                {"index": int(idx), "score": float(score)}
                for idx, score in zip(query_indices, query_scores)
            ]
        }
        results.append(result)
    return results

In [40]:
results = search(df['Вопрос'].tolist())

  0%|          | 0/10 [00:00<?, ?it/s]


ValueError: too many values to unpack (expected 2)

In [ ]:

    
    retriever = RetrieverSystem()
    retriever.build_index(df)
    
    
    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2)